# InsightForge – AI-Powered Business Intelligence Assistant

## 1. Setup

In [1]:
# Setup for single-notebook RAG 

import os
from pathlib import Path
from typing import List, Dict, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# OpenAI + Chroma (vector DB)
try:
    from openai import OpenAI
except ImportError:
    raise ImportError(
        "The 'openai' package is not installed. "
        "Install it with: pip install openai"
    )

try:
    import chromadb
    from chromadb.config import Settings
except ImportError:
    raise ImportError(
        "The 'chromadb' package is not installed. "
        "Install it with: pip install chromadb"
    )

# Project paths
PROJECT_ROOT = Path(".").resolve()
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_RAW.mkdir(parents=True, exist_ok=True)

CHROMA_DIR = PROJECT_ROOT / "chroma_db_insightforge"
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

# OpenAI client (API key via env variable)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError(
        "OPENAI_API_KEY environment variable is not set.\n"
        "Set it in your system or in the notebook before running:\n"
        "import os; os.environ['OPENAI_API_KEY'] = 'sk-...' "
    )

client = OpenAI(api_key=api_key)

print("Setup complete")
print("Project root:", PROJECT_ROOT)
print("Raw data folder:", DATA_RAW)
print("Chroma DB folder:", CHROMA_DIR)


Setup complete
Project root: C:\Users\mehra\InsightForge
Raw data folder: C:\Users\mehra\InsightForge\data\raw
Chroma DB folder: C:\Users\mehra\InsightForge\chroma_db_insightforge


## Step 2 – Load business data

In [2]:
# List available CSV files in data/raw
csv_files = list(DATA_RAW.glob("*.csv"))

print(f"Found {len(csv_files)} CSV file(s) in {DATA_RAW}:")
for f in csv_files:
    print(" -", f.name)

if not csv_files:
    print(
        "\nNo CSVs found yet. "
        "Copy your data files (e.g., sales.csv, customers.csv) into data/raw/ and rerun this cell."
    )


Found 1 CSV file(s) in C:\Users\mehra\InsightForge\data\raw:
 - sales_data.csv


In [3]:
def load_datasets(raw_dir: Path) -> Dict[str, pd.DataFrame]:
    # Load all CSV files from data/raw into a dict of DataFrames.
    # Keys are lowercase file stems, e.g. 'sales' for 'sales.csv'.
    
    datasets: Dict[str, pd.DataFrame] = {}
    for f in raw_dir.glob("*.csv"):
        key = f.stem.lower()
        df = pd.read_csv(f)
        datasets[key] = df
        print(f"Loaded '{key}': {df.shape[0]} rows, {df.shape[1]} columns")
    return datasets


datasets = load_datasets(DATA_RAW)

print("\nAvailable dataset keys:", list(datasets.keys()))


Loaded 'sales_data': 2500 rows, 7 columns

Available dataset keys: ['sales_data']


In [4]:
for name, df in datasets.items():
    print(f"\n===== {name} (first 5 rows) =====")
    display(df.head())



===== sales_data (first 5 rows) =====


,Date,Product,Region,Sales,Customer_Age,Customer_Gender,Customer_Satisfaction
0,2022-01-01,Widget C,South,786,26,Male,2.874407
1,2022-01-02,Widget D,East,850,29,Male,3.365205
2,2022-01-03,Widget A,North,871,40,Female,4.547364
3,2022-01-04,Widget C,South,464,31,Male,4.555420
4,2022-01-05,Widget C,South,262,50,Female,3.982935


## Step 3 – Configure schema & basic analytics helpers

In [5]:
datasets["sales_data"].columns


Index(['Date', 'Product', 'Region', 'Sales', 'Customer_Age', 'Customer_Gender',
       'Customer_Satisfaction'],
      dtype='object')

In [6]:
CONFIG = {
    "sales_table": "sales_data",   # matches datasets["sales_data"]
    "date_col": "Date",            # exact column name (case-sensitive)
    "value_col": "Sales",          # numeric sales amount
    "region_col": "Region",        # region name
    "product_col": "Product",      # product name
}


def get_sales_df(datasets: Dict[str, pd.DataFrame], config: Dict[str, str]) -> pd.DataFrame:
    # Returns the main sales DataFrame with parsed date column.
    key = config["sales_table"]
    if key not in datasets:
        raise KeyError(
            f"CONFIG['sales_table'] = '{key}', but this key is not in datasets: {list(datasets.keys())}"
        )

    df = datasets[key].copy()

    date_col = config["date_col"]
    if date_col not in df.columns:
        raise KeyError(
            f"CONFIG['date_col'] = '{date_col}', but this column is not in {key} columns: {list(df.columns)}"
        )

    # Parse dates
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    # Drop rows with invalid/missing dates
    df = df.dropna(subset=[date_col])

    # Optional: sort by date
    df = df.sort_values(by=date_col)

    print(f"Sales table '{key}': {df.shape[0]} rows, {df.shape[1]} columns")
    return df


sales_df = get_sales_df(datasets, CONFIG)
sales_df.head()


Sales table 'sales_data': 2500 rows, 7 columns


,Date,Product,Region,Sales,Customer_Age,Customer_Gender,Customer_Satisfaction
0,2022-01-01,Widget C,South,786,26,Male,2.874407
1,2022-01-02,Widget D,East,850,29,Male,3.365205
2,2022-01-03,Widget A,North,871,40,Female,4.547364
3,2022-01-04,Widget C,South,464,31,Male,4.555420
4,2022-01-05,Widget C,South,262,50,Female,3.982935


## Step 4 – Build a mini knowledge base & QA


In [7]:
def build_text_summaries(
    sales_df: pd.DataFrame,
    config: Dict[str, str]
) -> list[dict]:
    # Build a list of small text summaries (documents) from the sales data.
    # Each item is a dict with fields: id, text, metadata.

    date_col = config["date_col"]
    value_col = config["value_col"]
    region_col = config["region_col"]
    product_col = config["product_col"]

    docs: list[dict] = []

    # Overall summary
    total_sales = sales_df[value_col].sum()
    start_date = sales_df[date_col].min().date()
    end_date = sales_df[date_col].max().date()
    overall_text = (
        f"Overall sales between {start_date} and {end_date} "
        f"total {total_sales:,.2f} units or currency units."
    )
    docs.append({
        "id": "overall",
        "text": overall_text,
        "metadata": {"type": "overall"}
    })

    # Sales by region
    sales_by_region = (
        sales_df.groupby(region_col)[value_col]
        .sum()
        .reset_index()
        .sort_values(by=value_col, ascending=False)
    )
    for i, row in sales_by_region.iterrows():
        txt = (
            f"Region '{row[region_col]}' has total sales of "
            f"{row[value_col]:,.2f}."
        )
        docs.append({
            "id": f"region_{i}",
            "text": txt,
            "metadata": {
                "type": "region",
                "region": str(row[region_col])
            }
        })

    # Sales by product
    sales_by_product = (
        sales_df.groupby(product_col)[value_col]
        .sum()
        .reset_index()
        .sort_values(by=value_col, ascending=False)
    )
    for i, row in sales_by_product.iterrows():
        txt = (
            f"Product '{row[product_col]}' has total sales of "
            f"{row[value_col]:,.2f}."
        )
        docs.append({
            "id": f"product_{i}",
            "text": txt,
            "metadata": {
                "type": "product",
                "product": str(row[product_col])
            }
        })

    # Sales over time (monthly)
    monthly = (
        sales_df
        .set_index(date_col)
        .resample("M")[value_col]
        .sum()
        .reset_index()
    )
    for i, row in monthly.iterrows():
        d = row[date_col].date()
        txt = f"In {d}, total sales were {row[value_col]:,.2f}."
        docs.append({
            "id": f"month_{i}",
            "text": txt,
            "metadata": {
                "type": "month",
                "date": str(d)
            }
        })

    print(f"Built {len(docs)} text summaries.")
    return docs


kb_docs = build_text_summaries(sales_df, CONFIG)
kb_docs[:3]


Built 92 text summaries.


C:\Users\mehra\AppData\Local\Temp\ipykernel_26620\784388248.py:75: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")[value_col]


[{'id': 'overall',
  'text': 'Overall sales between 2022-01-01 and 2028-11-04 total 1,383,220.00 units or currency units.',
  'metadata': {'type': 'overall'}},
 {'id': 'region_3',
  'text': "Region 'West' has total sales of 361,383.00.",
  'metadata': {'type': 'region', 'region': 'West'}},
 {'id': 'region_1',
  'text': "Region 'North' has total sales of 353,025.00.",
  'metadata': {'type': 'region', 'region': 'North'}}]

In [8]:
# Create a persistent Chroma client
chroma_client = chromadb.PersistentClient(
    path=str(CHROMA_DIR),
    settings=Settings(anonymized_telemetry=False)
)

collection_name = "insightforge_kb"

existing_collections = [c.name for c in chroma_client.list_collections()]
if collection_name in existing_collections:
    chroma_client.delete_collection(name=collection_name)

# Create a fresh empty collection
collection = chroma_client.create_collection(name=collection_name)

print(f"Fresh Chroma collection '{collection_name}' created.")


def embed_texts(texts: list[str]) -> list[list[float]]:
    """
    Use OpenAI embedding model to embed a list of texts.
    """
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return [d.embedding for d in response.data]


# Add KB docs into Chroma
ids = [d["id"] for d in kb_docs]
texts = [d["text"] for d in kb_docs]
metas = [d["metadata"] for d in kb_docs]

embeddings = embed_texts(texts)

collection.add(
    ids=ids,
    documents=texts,
    metadatas=metas,
    embeddings=embeddings,
)

print(f"Stored {len(ids)} docs in Chroma collection '{collection_name}'.")


Fresh Chroma collection 'insightforge_kb' created.
Stored 92 docs in Chroma collection 'insightforge_kb'.


In [9]:
def retrieve_context(query: str, n_results: int = 5) -> list[str]:
    # Retrieve top-n relevant documents for the query from Chroma.
    q_emb = embed_texts([query])[0]

    results = collection.query(
        query_embeddings=[q_emb],  # <-- use embeddings, NOT query_texts
        n_results=n_results,
    )

    docs = results.get("documents", [[]])[0]
    return docs



SYSTEM_PROMPT = (
    "You are an AI business intelligence assistant. "
    "You must answer strictly based on the provided context, "
    "which contains sales summaries for products, regions, and time periods. "
    "If the answer is not in the context, say you don't know. "
    "Be concise and business-friendly."
)


def answer_question(query: str, n_results: int = 5) -> str:
    context_docs = retrieve_context(query, n_results=n_results)
    context_text = "\n\n".join(context_docs)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": (
                f"Context:\n{context_text}\n\n"
                f"Question: {query}\n\n"
                "Answer based only on the context above."
            ),
        },
    ]

    resp = client.chat.completions.create(
        model="gpt-4.1-mini",  # or "gpt-4.1" if you like
        messages=messages,
        temperature=0.2,
    )

    return resp.choices[0].message.content.strip()


In [10]:
test_questions = [
    "Which region has the highest total sales?",
    "Which product sells the most overall?",
    "How do sales change over time?",
]

for q in test_questions:
    print(f"\nQ: {q}")
    print("A:", answer_question(q))



Q: Which region has the highest total sales?
A: The West region has the highest total sales with 361,383.00.

Q: Which product sells the most overall?
A: Product 'Widget A' sells the most overall with total sales of 375,235.00.

Q: How do sales change over time?
A: Sales fluctuate over time, with values such as 15,597 in August 2025, rising to 18,695 in October 2025, then varying to 20,387 in April 2028 and 16,019 in July 2028. Overall, there is no clear consistent upward or downward trend based on the provided data points.


## Step 5 – Interactive QA chat inside the notebook

In [11]:
def debug_retrieval(query: str, n_results: int = 5):

    q_emb = embed_texts([query])[0]

    results = collection.query(
        query_embeddings=[q_emb],
        n_results=n_results,
    )
    docs = results.get("documents", [[]])[0]
    metas = results.get("metadatas", [[]])[0]

    print(f"Retrieved {len(docs)} docs for query: {query!r}\n")
    for i, (d, m) in enumerate(zip(docs, metas), start=1):
        print(f"--- Doc {i} ---")
        print("Metadata:", m)
        print("Text:", d)
        print()


In [12]:
def chat_loop():
    # Terminal-style chat loop
    # Type 'exit' or 'quit' to stop.
    print("InsightForge BI Assistant")
    print("Type your question (or 'exit' to quit).")
    print("-" * 50)

    while True:
        try:
            user_q = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nExiting chat.")
            break

        if user_q.lower() in {"exit", "quit"}:
            print("Exiting chat.")
            break

        if not user_q:
            continue

        try:
            answer = answer_question(user_q, n_results=5)
            print("Assistant:", answer)
            print("-" * 50)
        except Exception as e:
            print("Error while answering:", e)
            print("-" * 50)


# Start the chat (run this cell to interact)
chat_loop()


InsightForge BI Assistant
Type your question (or 'exit' to quit).
--------------------------------------------------


You:  What were the total sales during the period covered in the dataset?


Assistant: The total sales during the period covered in the dataset were 88,500.00.
--------------------------------------------------


You:  Which region has the highest total sales?


Assistant: The West region has the highest total sales with 361,383.00 units or currency units.
--------------------------------------------------


...

Assistant: The North region has higher total sales (353,025.00) compared to the South region (348,516.00).
--------------------------------------------------


You:  What can you tell me about customer satisfaction across products or regions?


Assistant: The provided context does not include any information about customer satisfaction across products or regions.
--------------------------------------------------


You:  exit


Exiting chat.


## Step 6 – Simple evaluation of the BI assistant

In [13]:
# Compute ground-truth stats from sales_df

region_col = CONFIG["region_col"]
product_col = CONFIG["product_col"]
value_col = CONFIG["value_col"]
date_col = CONFIG["date_col"]

# Sales by region
sales_by_region = (
    sales_df.groupby(region_col)[value_col]
    .sum()
    .reset_index()
    .sort_values(by=value_col, ascending=False)
)

# Sales by product
sales_by_product = (
    sales_df.groupby(product_col)[value_col]
    .sum()
    .reset_index()
    .sort_values(by=value_col, ascending=False)
)

top_region = sales_by_region.iloc[0][region_col]
top_region_value = sales_by_region.iloc[0][value_col]

top_product = sales_by_product.iloc[0][product_col]
top_product_value = sales_by_product.iloc[0][value_col]

print("Top region:", top_region, "| Sales:", top_region_value)
print("Top product:", top_product, "| Sales:", top_product_value)

# Monthly sales for trend
monthly = (
    sales_df
    .set_index(date_col)
    .resample("M")[value_col]
    .sum()
    .reset_index()
)

first_val = monthly[value_col].iloc[0]
last_val = monthly[value_col].iloc[-1]

if last_val > first_val * 1.05:        # >5% up
    trend = "increasing"
elif last_val < first_val * 0.95:      # >5% down
    trend = "decreasing"
else:
    trend = "roughly stable"

print("Overall sales trend:", trend)


Top region: West | Sales: 361383
Top product: Widget A | Sales: 375235
Overall sales trend: decreasing


C:\Users\mehra\AppData\Local\Temp\ipykernel_26620\39074439.py:37: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")[value_col]


In [14]:
# Build evaluation set with REAL reference answers

eval_questions = [
    {
        "id": 1,
        "question": "Which region has the highest total sales?",
        "reference_answer": f"The region with the highest total sales is {top_region}."
    },
    {
        "id": 2,
        "question": "Which product has the highest total sales?",
        "reference_answer": f"The product with the highest total sales is {top_product}."
    },
    {
        "id": 3,
        "question": "In general, are sales increasing or decreasing over time?",
        "reference_answer": f"Overall, sales are {trend} over time."
    },
]

eval_questions


[{'id': 1,
  'question': 'Which region has the highest total sales?',
  'reference_answer': 'The region with the highest total sales is West.'},
 {'id': 2,
  'question': 'Which product has the highest total sales?',
  'reference_answer': 'The product with the highest total sales is Widget A.'},
 {'id': 3,
  'question': 'In general, are sales increasing or decreasing over time?',
  'reference_answer': 'Overall, sales are decreasing over time.'}]

In [15]:
# Grading function using GPT as a judge

import json

def grade_answer_with_gpt(question: str, reference: str, model_answer: str) -> dict:
    """
    Use GPT to grade the model's answer against a reference.
    Returns a dict with grade ('CORRECT' or 'INCORRECT') and explanation.
    """
    system_prompt = (
        "You are a strict grader for a business intelligence assistant.\n"
        "Given a question, a reference answer (ground truth), and the model's answer, "
        "decide whether the model's answer is overall CORRECT or INCORRECT.\n"
        "Ignore minor wording differences and focus on factual correctness.\n"
        "Reply ONLY in valid JSON with fields 'grade' and 'explanation'."
    )

    user_content = f"""
Question:
{question}

Reference answer (ground truth):
{reference}

Model's answer:
{model_answer}
"""

    resp = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content},
        ],
        temperature=0.0,
    )

    raw = resp.choices[0].message.content.strip()

    try:
        result = json.loads(raw)
        grade = result.get("grade", "UNKNOWN").upper()
        explanation = result.get("explanation", "")
    except Exception:
        grade = "UNKNOWN"
        explanation = f"Could not parse grader response: {raw}"

    return {"grade": grade, "explanation": explanation}


In [16]:
# Run evaluation loop and compute accuracy

results = []

for item in eval_questions:
    q = item["question"]
    ref = item["reference_answer"]

    print(f"\n=== Evaluating Q{item['id']}: {q}")
    model_answer = answer_question(q)
    print("Model answer:", model_answer)

    grading = grade_answer_with_gpt(q, ref, model_answer)
    print("Grade:", grading["grade"])
    print("Explanation:", grading["explanation"])

    results.append({
        "id": item["id"],
        "question": q,
        "reference_answer": ref,
        "model_answer": model_answer,
        "grade": grading["grade"],
        "explanation": grading["explanation"],
    })

eval_df = pd.DataFrame(results)
display(eval_df)

accuracy = (eval_df["grade"] == "CORRECT").mean()
print(f"\nOverall accuracy on eval set: {accuracy:.2f}")



=== Evaluating Q1: Which region has the highest total sales?
Model answer: The West region has the highest total sales with 361,383.00.
Grade: CORRECT
Explanation: The model's answer correctly identifies the West region as having the highest total sales, consistent with the reference answer. The inclusion of the sales figure adds useful detail but does not affect correctness.

=== Evaluating Q2: Which product has the highest total sales?
Model answer: Product 'Widget A' has the highest total sales with 375,235.00.
Grade: CORRECT
Explanation: The model's answer correctly identifies 'Widget A' as the product with the highest total sales, matching the reference answer. The additional sales figure provided does not contradict the reference and supports the correctness.

=== Evaluating Q3: In general, are sales increasing or decreasing over time?
Model answer: Based on the provided data, sales appear to be generally increasing over time, with totals rising from 15,897.00 in July 2023 to 19

,id,question,reference_answer,model_answer,grade,explanation
0,1,Which region has the highest total sales?,The region with the highest total sales is West.,The West region has the highest total sales wi...,CORRECT,The model's answer correctly identifies the We...
1,2,Which product has the highest total sales?,The product with the highest total sales is Wi...,Product 'Widget A' has the highest total sales...,CORRECT,The model's answer correctly identifies 'Widge...
2,3,"In general, are sales increasing or decreasing...","Overall, sales are decreasing over time.","Based on the provided data, sales appear to be...",INCORRECT,The reference answer states that sales are dec...



Overall accuracy on eval set: 0.67


In [17]:
accuracy = (eval_df["grade"] == "CORRECT").mean()
print(f"Overall accuracy on eval set: {accuracy:.2f}")


Overall accuracy on eval set: 0.67


## Step 7 – System Summary Report

In [21]:
# Summary

# Basic dataset info
n_rows = len(sales_df)
n_cols = sales_df.shape[1]
start_date = monthly[CONFIG["date_col"]].iloc[0].date()
end_date = monthly[CONFIG["date_col"]].iloc[-1].date()
total_sales = sales_df[CONFIG["value_col"]].sum()

# Evaluation accuracy (already computed in Step 6)
accuracy = (eval_df["grade"] == "CORRECT").mean() if "grade" in eval_df.columns else float("nan")

summary_text = f"""
### InsightForge BI Assistant – Summary

**Data overview**

- Number of rows: **{n_rows}**
- Number of columns: **{n_cols}**
- Time span: **{start_date}** to **{end_date}**
- Total sales (all regions and products): **{total_sales:,.2f}**

**Key descriptive insights**

- Region with highest total sales: **{top_region}** (≈ **{top_region_value:,.2f}** total sales)
- Product with highest total sales: **{top_product}** (≈ **{top_product_value:,.2f}** total sales)
- Overall sales trend over time: **{trend}**

**Assistant behavior**

- The assistant uses:
  - Pandas-based aggregations over the sales dataset
  - OpenAI embeddings (`text-embedding-3-small`) stored in a Chroma vector database
  - Retrieval-augmented generation (RAG) to answer questions based only on stored summaries
- Answers are constrained to the retrieved context and are designed to be concise and business-focused.

**Evaluation**

- Number of evaluation questions: **{len(eval_df)}**
- Accuracy (fraction graded as CORRECT by GPT-based judge): **{accuracy:.2f}**

This configuration demonstrates how a lightweight RAG pipeline can turn a single tabular sales dataset into an interactive BI assistant that
supports natural-language queries while remaining grounded in the underlying data.
"""

print(summary_text)



### InsightForge BI Assistant – Summary

**Data overview**

- Number of rows: **2500**
- Number of columns: **7**
- Time span: **2022-01-31** to **2028-11-30**
- Total sales (all regions and products): **1,383,220.00**

**Key descriptive insights**

- Region with highest total sales: **West** (≈ **361,383.00** total sales)
- Product with highest total sales: **Widget A** (≈ **375,235.00** total sales)
- Overall sales trend over time: **decreasing**

**Assistant behavior**

- The assistant uses:
  - Pandas-based aggregations over the sales dataset
  - OpenAI embeddings (`text-embedding-3-small`) stored in a Chroma vector database
  - Retrieval-augmented generation (RAG) to answer questions based only on stored summaries
- Answers are constrained to the retrieved context and are designed to be concise and business-focused.

**Evaluation**

- Number of evaluation questions: **3**
- Accuracy (fraction graded as CORRECT by GPT-based judge): **0.67**

This configuration demonstrates how a l